## Finding Hubs and Zones in Pricing Data

Figuring out how to find Hubs or Zones and then query those location names are common first tasks when working with LMP data. This notebook shows some basic queries to help users of the gridstatusio API determine the correct location_type filters to use in order to get the correct locations.

In [1]:
from gridstatusio import GridStatusClient

client = GridStatusClient()

GridStatusClient(host=https://api.gridstatus.io/v1)

## SPP

Two hubs in the lmp data. The by_bus dataset does not include hubs.

In [2]:
df_spp = client.get_dataset(
    dataset="spp_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)
df_spp

Fetching Page 1...Done in 0.33 seconds. 

Total number of rows: 2


,interval_start_utc,interval_end_utc,market,location,location_type,pnode,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,SPPNORTH_HUB,Hub,SPPNORTH_H,12.4283,11.2583,1.2922,-0.1222
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,SPPSOUTH_HUB,Hub,SPPSOUTH_H,-8.7672,11.2584,-19.2451,-0.7805


## MISO
MISO labels many nodes as "Hub" in the raw data, but there are a much smaller subset that are actually state-level Hubs which people look for. To get down to those we filter on the `location_type` of "Hub" and then further filter the dataset based on location names.

In [3]:
df_miso = client.get_dataset(
    dataset="miso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Hub",
)

df_miso_hubs = df_miso[df_miso["location"].str.contains("HUB")]
df_miso_hubs

Fetching Page 1...Done in 0.26 seconds. 

Total number of rows: 406


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
83,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ARKANSAS.HUB,Hub,19.8600,19.0900,1.5600,-0.7900
264,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ILLINOIS.HUB,Hub,9.6400,19.0900,-9.1100,-0.3400
265,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,INDIANA.HUB,Hub,27.1900,19.0900,7.1900,0.9100
279,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,LOUISIANA.HUB,Hub,21.1900,19.0900,2.0200,0.0800
310,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MICHIGAN.HUB,Hub,26.6000,19.0900,6.1000,1.4100
311,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MINN.HUB,Hub,12.1500,19.0900,-6.4500,-0.4900
319,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MS.HUB,Hub,21.0000,19.0900,2.1300,-0.2200
388,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TEXAS.HUB,Hub,20.4900,19.0900,1.7000,-0.3000


## ERCOT
ERCOT has 5 Hubs and also publishes two averages, one each for Buses and Hubs.

In [4]:
df_ercot_hubs = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_ercot_hubs

Fetching Page 1...Done in 0.21 seconds. 

Total number of rows: 7


,interval_start_utc,interval_end_utc,location,location_type,market,spp
0,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_BUSAVG,Trading Hub,REAL_TIME_15_MIN,19.9600
1,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_HOUSTON,Trading Hub,REAL_TIME_15_MIN,20.3000
2,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_HUBAVG,Trading Hub,REAL_TIME_15_MIN,20.1300
3,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_NORTH,Trading Hub,REAL_TIME_15_MIN,19.8600
4,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_PAN,Trading Hub,REAL_TIME_15_MIN,11.5700
5,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_SOUTH,Trading Hub,REAL_TIME_15_MIN,19.3900
6,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,HB_WEST,Trading Hub,REAL_TIME_15_MIN,20.9800


ERCOT also publishes data for several load zones

In [5]:
df_ercot_zones = client.get_dataset(
    dataset="ercot_spp_real_time_15_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:15",
    filter_column="location_type",
    filter_value="Load Zone",
)
df_ercot_zones

Fetching Page 1...Done in 0.26 seconds. 

Total number of rows: 8


,interval_start_utc,interval_end_utc,location,location_type,market,spp
0,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_AEN,Load Zone,REAL_TIME_15_MIN,20.8600
1,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_CPS,Load Zone,REAL_TIME_15_MIN,21.3100
2,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_HOUSTON,Load Zone,REAL_TIME_15_MIN,20.3100
3,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_LCRA,Load Zone,REAL_TIME_15_MIN,20.4900
4,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_NORTH,Load Zone,REAL_TIME_15_MIN,19.8300
5,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_RAYBN,Load Zone,REAL_TIME_15_MIN,19.7200
6,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_SOUTH,Load Zone,REAL_TIME_15_MIN,23.0300
7,2024-04-01 00:00:00+00:00,2024-04-01 00:15:00+00:00,LZ_WEST,Load Zone,REAL_TIME_15_MIN,22.9300


## CAISO
CAISO has 3 Trading Hubs in their LMP data.

In [6]:
df_caiso = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Trading Hub",
)
df_caiso

Fetching Page 1...Done in 1.1 seconds. 

Total number of rows: 3


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_NP15_GEN-APND,Trading Hub,-3.9957,-3.3000,-0.7050,0.0092
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_SP15_GEN-APND,Trading Hub,-3.1620,-3.3000,0.0000,0.1379
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,TH_ZP26_GEN-APND,Trading Hub,-3.1469,-3.3000,0.0000,0.1531


CAISO also publishes data for several DLAPs (Default Load Aggregation Points) which is the node at which all bids for demand are submitted and settled

In [7]:
df_caiso_dlap = client.get_dataset(
    dataset="caiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="DLAP",
)
df_caiso_dlap

Fetching Page 1...Done in 1.96 seconds. 

Total number of rows: 6


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PACE_NPM-APND,DLAP,-3.2984,-3.3000,0.0012,0.0003
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PACW_NPM-APND,DLAP,-3.2974,-3.3000,0.0012,0.0013
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_PGAE-APND,DLAP,-4.2795,-3.3000,-0.9400,-0.0396
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_SCE-APND,DLAP,-3.2369,-3.3000,0.0000,0.0630
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_SDGE-APND,DLAP,-3.2442,-3.3000,0.0000,0.0558
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DLAP_VEA-APND,DLAP,-3.0977,-3.3000,0.0000,0.2023


## PJM Hubs
As the largest market in the US, PJM also has a higher number of hubs than most markets.

In [8]:
df_pjm = client.get_dataset(
    dataset="pjm_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_pjm

Fetching Page 1...Done in 0.27 seconds. 

Total number of rows: 12


,interval_start_utc,interval_end_utc,market,location,location_id,location_short_name,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,AEP-DAYTON HUB,34497127,AEP-DAYTON HUB,HUB,31.6800,31.2600,0.4100,0.0100
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,AEP GEN HUB,34497125,AEP GEN HUB,HUB,31.4500,31.2600,0.7500,-0.5600
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,ATSI GEN HUB,116013751,ATSI GEN HUB,HUB,31.4600,31.2600,0.4600,-0.2600
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CHICAGO GEN HUB,33092311,CHICAGO GEN HUB,HUB,24.6900,31.2600,-5.1800,-1.3900
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CHICAGO HUB,33092313,CHICAGO HUB,HUB,25.3200,31.2600,-4.9400,-1.0000
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DOMINION HUB,35010337,DOMINION HUB,HUB,31.9500,31.2600,0.7400,-0.0500
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,EASTERN HUB,51217,EASTERN HUB,HUB,32.1400,31.2600,0.6200,0.2600
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,NEW JERSEY HUB,4669664,NEW JERSEY HUB,HUB,31.8900,31.2600,0.6000,0.0300
8,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,N ILLINOIS HUB,33092315,N ILLINOIS HUB,HUB,25.1100,31.2600,-5.0100,-1.1400
9,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,OHIO HUB,34497151,OHIO HUB,HUB,31.6300,31.2600,0.3000,0.0700


## NYISO
NYISO doesn't calculate and publish hubs as a separate category. The market doesn't price at every bus, only at generators and the zonal level, so we will get the Zone names instead. This list includes external regions such as Ontario (OH) and Quebec (HQ).

In [9]:
df_nyiso = client.get_dataset(
    dataset="nyiso_lmp_real_time_5_min",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="Zone",
)
df_nyiso

Fetching Page 1...Done in 0.17 seconds. 

Total number of rows: 15


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CAPITL,Zone,20.1300,19.5800,-0.0000,0.5500
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,CENTRL,Zone,19.6000,19.5800,-0.0000,0.0200
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,DUNWOD,Zone,20.5600,19.5800,-0.0000,0.9800
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,GENESE,Zone,19.3100,19.5800,-0.0000,-0.2700
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,H Q,Zone,19.5800,19.5800,-0.0000,0.0000
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,HUD VL,Zone,20.4800,19.5800,-0.0000,0.9000
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,LONGIL,Zone,20.6600,19.5800,-0.0000,1.0800
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MHK VL,Zone,20.1500,19.5800,-0.0000,0.5700
8,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,MILLWD,Zone,20.5000,19.5800,-0.0000,0.9200
9,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,NORTH,Zone,19.6200,19.5800,-0.0000,0.0400


## ISO-NE
ISO-NE only has a single internal Hub, so we will also query the Load Zones.

In [10]:
df_isone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min_prelim",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="HUB",
)
df_isone

Fetching Page 1...Done in 0.28 seconds. 

Total number of rows: 1


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.H.INTERNAL_HUB,HUB,34.9100,34.6900,0.0500,0.1700


In [11]:
df_isone_load_zone = client.get_dataset(
    dataset="isone_lmp_real_time_5_min_prelim",
    start="2024-04-01 00:00",
    end="2024-04-01 00:05",
    filter_column="location_type",
    filter_value="LOAD ZONE",
)
df_isone_load_zone

Fetching Page 1...Done in 0.54 seconds. 

Total number of rows: 8


,interval_start_utc,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.CONNECTICUT,LOAD ZONE,34.0100,34.6900,0.0500,-0.7300
1,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.MAINE,LOAD ZONE,34.0100,34.6900,-0.3900,-0.2900
2,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.NEMASSBOST,LOAD ZONE,35.2700,34.6900,0.0500,0.5300
3,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.NEWHAMPSHIRE,LOAD ZONE,34.9800,34.6900,0.0500,0.2400
4,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.RHODEISLAND,LOAD ZONE,34.8100,34.6900,0.0500,0.0700
5,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.SEMASS,LOAD ZONE,35.2000,34.6900,0.0500,0.4600
6,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.VERMONT,LOAD ZONE,34.6300,34.6900,0.0500,-0.1100
7,2024-04-01 00:00:00+00:00,2024-04-01 00:05:00+00:00,REAL_TIME_5_MIN,.Z.WCMASS,LOAD ZONE,34.8400,34.6900,0.0500,0.1000


Now let's print out the lists of locations from each market.

In [12]:
print("\nCAISO Trading Hub:", list(df_caiso["location"].unique()))
print("CAISO DLAP:", list(df_caiso_dlap["location"].unique()), "\n")
print("ERCOT Trading Hub:", list(df_ercot_hubs["location"].unique()))
print("ERCOT Load Zone:", list(df_ercot_zones["location"].unique()), "\n")
print("SPP:", list(df_spp["location"].unique()), "\n")
print("MISO:", list(df_miso_hubs["location"].unique()), "\n")
print("PJM:", list(df_pjm["location"].unique()), "\n")
print("NYISO:", list(df_nyiso["location"].unique()), "\n")
print("ISONE Hub:", list(df_isone["location"].unique()))
print("ISONE Load Zone:", list(df_isone_load_zone["location"].unique()), "\n")


CAISO Trading Hub: ['TH_NP15_GEN-APND', 'TH_SP15_GEN-APND', 'TH_ZP26_GEN-APND']
CAISO DLAP: ['DLAP_PACE_NPM-APND', 'DLAP_PACW_NPM-APND', 'DLAP_PGAE-APND', 'DLAP_SCE-APND', 'DLAP_SDGE-APND', 'DLAP_VEA-APND'] 

ERCOT Trading Hub: ['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_PAN', 'HB_SOUTH', 'HB_WEST']
ERCOT Load Zone: ['LZ_AEN', 'LZ_CPS', 'LZ_HOUSTON', 'LZ_LCRA', 'LZ_NORTH', 'LZ_RAYBN', 'LZ_SOUTH', 'LZ_WEST'] 

SPP: ['SPPNORTH_HUB', 'SPPSOUTH_HUB'] 

MISO: ['ARKANSAS.HUB', 'ILLINOIS.HUB', 'INDIANA.HUB', 'LOUISIANA.HUB', 'MICHIGAN.HUB', 'MINN.HUB', 'MS.HUB', 'TEXAS.HUB'] 

PJM: ['AEP-DAYTON HUB', 'AEP GEN HUB', 'ATSI GEN HUB', 'CHICAGO GEN HUB', 'CHICAGO HUB', 'DOMINION HUB', 'EASTERN HUB', 'NEW JERSEY HUB', 'N ILLINOIS HUB', 'OHIO HUB', 'WESTERN HUB', 'WEST INT HUB'] 

NYISO: ['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'H Q', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'NORTH', 'NPX', 'N.Y.C.', 'O H', 'PJM', 'WEST'] 

ISONE Hub: ['.H.INTERNAL_HUB']
ISONE Load Zone: ['.Z.CONNECTICU